In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
from preprocessing_helper import *

### 1. Get directories related information and load in MoonBoard 2016 data

In [ ]:
cwd = os.getcwd()
parent_wd=os.path.dirname(cwd)
raw_data_path = os.path.join(parent_wd,'raw_data','moonGen_scrape_2016_final')
hold_feature_path = os.path.join(parent_wd,'raw_data','HoldFeature2016.xlsx')
print(raw_data_path)

In [ ]:
with open(raw_data_path, 'rb') as f:
    MoonBoard_2016_raw = pickle.load(f)

In [ ]:
features = pd.read_excel(hold_feature_path, dtype=str)

# convert features from pd dataframe to dictionary
feature_dict = {}
for index in features.index:
    feature_item = features.loc[index]
    feature_dict[(int(feature_item['X_coord']), int(feature_item['Y_coord']))] = np.array(
        list(feature_item['Difficulties'])).astype(int)

### 2. Separate data into different categories:
- no user ratings
- benchmarked with user ratings
- non-benchmarked with user ratings

In [ ]:
grade_map = get_grade_map()

In [ ]:
save_path = os.path.join(cwd,'processed_data_xy_mode')
output_xy_mode = classify_and_reorganize_data(MoonBoard_2016_raw, save_path, delta_xy_mode = False)

### 3. Data Preprocessing with BetaMove

In [ ]:
raw_data_forseq_path = os.path.join(parent_wd,'preprocessing','processed_data_xy_mode')
with open(raw_data_forseq_path, 'rb') as f:
    MoonBoard_2016_raw_forseq = pickle.load(f)

In [ ]:
X_dict_merge = {**MoonBoard_2016_raw_forseq['X_dict_benchmark_withgrade'], 
                **MoonBoard_2016_raw_forseq['X_dict_benchmark_nograde'], 
                **MoonBoard_2016_raw_forseq['X_dict_withgrade'], 
                **MoonBoard_2016_raw_forseq['X_dict_nograde']}
Y_dict_merge = {**MoonBoard_2016_raw_forseq['Y_dict_benchmark_withgrade'], 
                **MoonBoard_2016_raw_forseq['Y_dict_benchmark_nograde'], 
                **MoonBoard_2016_raw_forseq['Y_dict_withgrade'], 
                **MoonBoard_2016_raw_forseq['Y_dict_nograde']}

In [ ]:
save_path_seq = os.path.join(cwd,'processed_data_seq')
output_seq = generate_organized_sequence_data(raw_data = X_dict_merge, save_path = save_path_seq)
save_pickle(output_seq, save_path_seq)